In [19]:
# import libraries
import mobiledna.core.sessions
from mobiledna.core.appevents import Appevents
from mobiledna.core.sessions import Sessions
from mobiledna.core.notifications import Notifications

import pandas as pd

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import necessary files:

In [20]:
# app events from all waves
ae_wave2 = Appevents.load_data("/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/data/data_nervosity/wave_2/210310_nervocity_appevents_panelkitid.parquet")
ae_wave3 = Appevents.load_data("/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/data/data_nervosity/wave_3_2/210803_nervocity_appevents.csv", sep=';')

df_ae_wave2 = ae_wave2.__data__
df_ae_wave2.pop("panelkitid")
df_ae_wave3 = ae_wave3.__data__

# date ranges to filter on
df_dates = pd.read_csv("/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/data/data_nervosity/dates_nervo_allwaves.csv", sep=';')# date ranges to filter on
df_dates = pd.read_csv("/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/data/data_nervosity/dates_nervo_allwaves.csv", sep=';')

# mapping panelkit-id and mobileDNA-id
df_mapping = pd.read_csv("/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/data/data_nervosity/mobdna_mapping_w2w3.csv", sep=';')
df_mapping = df_mapping[['panelkit_id', 'mobdna_id','wave','campagne']].rename(columns={'mobdna_id': 'id'})


2021-10-22 12:00:44 - Recognized file type as <parquet>.
2021-10-22 12:00:44 - 'load' took 0.115 seconds to complete.
2021-10-22 12:00:44 - Recognized file type as <csv>.


/Users/simonperneel/Documents/Imec-mict/mobiledna_py/mobiledna/core/help.py:178: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  res = f(*args, **kwargs)


2021-10-22 12:00:47 - 'load' took 2.558 seconds to complete.


Put data from two waves together:

In [21]:
df_ae_all_waves = pd.concat([df_ae_wave2, df_ae_wave3])
print(df_ae_all_waves.shape)

(1897178, 17)


Add panelkit-id to mobileDNA-id:

In [22]:
id1 = df_ae_all_waves['id'].nunique()
df_ae_all_waves = df_ae_all_waves.merge(df_mapping, on='id')
df_ae_all_waves['date'] = df_ae_all_waves['startTime'].dt.date
df_ae_all_waves['date'] = pd.to_datetime(df_ae_all_waves['date'])
id2 = df_ae_all_waves['id'].nunique()

print(f"{(id1 - id2)} id's lost from {id1} id's after adding panelkit-id")

print(df_ae_all_waves.head())

109 id's lost from 420 id's after adding panelkit-id
                                     id     model     session               startTime                 endTime  notification  notificationId                 application  battery   latitude  longitude   studyKey        surveyId data_version  startDate    endDate  duration                           panelkit_id wave  campagne       date
0  0062be5c-4633-43ee-9785-2b164c1f1577  SM-A202F  1611987751 2021-01-30 07:22:38.553 2021-01-30 07:25:29.654         False               0         com.facebook.katana       44  50.985508   3.700484  nervocity  23121992inanjo          1.6 2021-01-30 2021-01-30   171.101  c32add21-2fe3-11eb-9dd4-cb04303e036e   w2         1 2021-01-30
1  0062be5c-4633-43ee-9785-2b164c1f1577  SM-A202F  1611987963 2021-01-30 07:26:06.485 2021-01-30 07:32:15.377         False               0       com.instagram.android       43  50.985460   3.700487  nervocity  23121992inanjo          1.6 2021-01-30 2021-01-30   368.892  c32ad

In [23]:
df_dates['date'] = pd.to_datetime(df_dates['date'])

all_entries = df_ae_all_waves.shape[0]
df_ae_filtered = df_ae_all_waves[df_ae_all_waves['date'].isin(df_dates['date'])]
df_ae_filtered.pop('date')
filtered_entries = df_ae_filtered.shape[0]

print(f"{(all_entries - filtered_entries)}/{all_entries} Appevents removed that were not in the date range")

878795/1488155 Appevents removed that were not in the date range


Export to csv

In [24]:
df_ae_filtered.to_csv("filtered_appevents.csv", sep=';', index=False)




